# Plain tdmclient interactive session

Communicating with a [Thymio II](https://thymio.org) robot is mostly the same from a Jupyter notebook or from the plain Python repl. You must also connect a Thymio II to your computer and start [Thymio Suite](https://www.thymio.org/program/). The main differences are that you interrupt execution by clicking the stop button (the little square button next to Run, below the menu) instead of typing control-C; and that you can use directly the `await` keyword. And of course, you can add titles, text, links, graphics, etc.

This notebook borrows largely from the help file of tdmclient.

First, make sure that tdmclient is installed for Jupyter:

In [ ]:
%pip install --upgrade --quiet tdmclient

Import the required class.

In [ ]:
from tdmclient import ClientAsync

Create a client object:

In [ ]:
client = ClientAsync()

If the TDM runs on your local computer, its address and port number will be obtained from [zeroconf](https://en.wikipedia.org/wiki/Zero-configuration_networking). You can check their value:

In [ ]:
client.tdm_addr

In [ ]:
client.tdm_port

The client will connect to the TDM which sends messages to us, such as one to announce the existence of a robot. The easiest way to receive and process them is to call asynchronous functions in such a way that their result is waited for. This can be done in a coroutine, a special function which is executed at the same time as other tasks your program must perform, with the `await` Python keyword; or handled by the helper function `aw`. In this notebook, we'll use mainly `await`. In the normal Python repl, `await` cannot be used directly; `aw` should be used instead.

Robots are associated to nodes. To get the first node once it's available (i.e. an object which refers to the first or only robot after having received and processed enough messages from the TDM to have this information), type

In [ ]:
node = await client.wait_for_node()

Lock the robot to change variables or run programs (make sure it isn't already used in Thymio Suite).

In [ ]:
await node.lock()

Compile and load an Aseba program:

In [ ]:
program = """
var on = 0  # 0=off, 1=on
timer.period[0] = 500

onevent timer0
    on = 1 - on  # "on = not on" with a syntax Aseba accepts
    leds.top = [32 * on, 32 * on, 0]
"""
r = await node.compile(program)

Run the program on the Thymio and observe its effect:

In [ ]:
await node.run()

Stop it:

In [ ]:
await node.stop()

Unlock the robot, so that it can be locked again by us in the next cell or by Thymio Suite:

In [ ]:
await node.unlock()

## Programs in async functions

Instead of calling asynchronous functions one by one, we can put a whole program in a single Python function, awaiting the result of each async function with the `await` keyword, and execute the function with `client.run_async_program`.

Define your program in an async function `prog`. There are a few points to note:
  - The node locking is done in a `with` construct, so that unlocking will be performed automatically at the end even if an error occurs. And `client.lock()` gives us directly the first node: no need to call `wait_for_node`.
  - Getting and setting Thymio variables is done via attributes of `node.v` using the same dotted syntax as used in the official documentation of the Thymio.
  - Thymio variable values are cached. When you read them, you get the last value which was received from the TDM. At the beginning, we await `node.wait_for_variables` to make sure the variable(s) we're interested has been received. When you set them, you must call `node.flush()` to send all your changes to the Thymio.
  - All variables are integers; that's why we use the integer division operator `//` instead of the floating-point division operator `/`.
  - No need to be too greedy, it's wise to slow down the loop by calling `client.sleep` with a sensible value. The meaning of _sensible_ depends on the speed of the connection. With the wireless dongle, it can be slow, especially with many robots in the same room.

In [ ]:
async def prog():
    with await client.lock() as node:
        await node.wait_for_variables({"prox.horizontal"})
        while True:
            prox_front = node.v.prox.horizontal[2]
            speed = -prox_front // 10
            node.v.motor.left.target = speed
            node.v.motor.right.target = speed
            node.flush()
            await client.sleep(0.1)

Run the program:

In [ ]:
await prog()

To stop the program, click the Stop button of Jupyter (_interrupt the kernel_).

We've used `with` to simplify the locking/unlocking of the node. We can also do it with the client object itself:

In [ ]:
with ClientAsync() as client:
    async def prog():
        with await client.lock() as node:
            await node.wait_for_variables({"prox.horizontal"})
            while True:
                prox_front = node.v.prox.horizontal[2]
                speed = -prox_front // 10
                node.v.motor.left.target = speed
                node.v.motor.right.target = speed
                node.flush()
                await client.sleep(0.1)
    await prog()